# Collision Avoidance - Live Demo(デモ)

In this notebook we'll use the model we trained to detect whether the robot is ``free`` or ``blocked`` to enable a collision avoidance behavior on the robot.  

(訳) このnotebookでは、ロボットが「フリー」か「ブロック」かを検出するトレーニング済みモデルを使用して、ロボットの衝突回避動作を有効にします。

## Load the trained model(学習済みモデルの読み込み)

We'll assumed that you've already downloaded the ``best_model.pth`` to your workstation as instructed in the training notebook.  Now, you should upload this model into this notebook's
directory by using the Jupyter Lab upload tool.  Once that's finished there should be a file named ``best_model.pth`` in this notebook's directory.  

(追加) ローカルの学習なので、すでに``best_model.pth``がローカルに存在している事を前提とします。

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model.  This should look very familiar from the training notebook.

(訳) Pytorch modelの初期化を下記コードでおこないます。

In [ ]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

Next, load the trained weights from the ``best_model.pth`` file that you uploaded

(訳) 次に、``best_model.pth``から、学習した重みを読み込みます。

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

(訳) 現在、モデルの重みは、CPUメモリーで実行されているので、下記コードでGPUに転送します。

In [ ]:
device = torch.device('cuda')
model = model.to(device)

### Create the preprocessing function(事前処理関数の作成)

We have now loaded our model, but there's a slight issue.  The format that we trained our model doesnt *exactly* match the format of the camera.  To do that, 
we need to do some *preprocessing*.  This involves the following steps

(訳) モデルを読み込みましたが、まだわずかな問題があります。学習済みモデルのフォーマットと、カメラの形式と*完全に*一致しません。これを解消するために、
いくつかの*前処理*を行う必要があります。これらは、下記の手順になります。

1. Convert from BGR to RGB
2. Convert from HWC layout to CHW layout
3. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
4. Transfer the data from CPU memory to GPU memory
5. Add a batch dimension

(訳)
1. BGRからRGBに変換
2. HWC layoutからCHW layoutに変換
3. トレーニング中に使ったのと同じパラメーターを使用して正規化します（カメラは[0、255]の範囲の値を提供し、ロードされた画像は[0、1]の範囲でトレーニングするため、255.0でスケーリングする必要があります
4. dataをCPUメモリからGPUメモリに転送します
5. バッチディメンションを追加する

In [ ]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Great! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

(訳) すばらしい、これでカメラ形式からneural networkのインプットのフォーマットに変換するための、pre-processing関数を定義する事ができました。

Now, let's start and display our camera.  You should be pretty familiar with this by now.  We'll also create a slider that will display the
probability that the robot is blocked.

(訳) カメラを起動し、画面に表示します。カメラの起動には、大分なれてきましたね。また、ロボットがブロックされる確率が表示されるスライダーを作成します。

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

We'll also create our robot instance which we'll need to drive the motors.

(訳) モーターを制御するためにrobotインスタンスを生成します。

In [ ]:
from jetbot import Robot

robot = Robot()

Next, we'll create a function that will get called whenever the camera's value changes.  This function will do the following steps

(訳) 次に、カメラの値が変更されるたびに呼び出される関数を作成します。この関数は下記手順で実行します。

1. Pre-process the camera image
2. Execute the neural network
3. While the neural network output indicates we're blocked, we'll turn left, otherwise we go forward.

(訳) 
1. カメラのイメージのPre-process
2. neural networkの実行
3. ニューラルネットワークの出力がブロックを示した場合は、左に曲がります。それ以外の場合は前進します。

In [ ]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.5:
        robot.forward(0.4)
    else:
        robot.left(0.4)
    
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to intialize

Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing. 

(訳)すばらしい、neural networkの実行関数が生成できたので、あとは処理をカメラと関連づける必要がある。

We accomplish that with the ``observe`` function.

(訳) それは `` observe``関数で実現します。

> WARNING: This code will move the robot!! Please make sure your robot has clearance.  The collision avoidance should work, but the neural
> network is only as good as the data it's trained on!

> (訳)
> 注意: このコードでロボットが動きだします。十分なスペースを確保してください。collision avoidanceが動くでしょう、neral networkは、訓練されたデータと同じくらいに動くはずです。

In [ ]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame.  Perhaps start by placing your robot on the ground and seeing what it does when it reaches an obstacle.

(訳) 驚くばかり！ロボットが接続されている場合、新しいカメラフレームごとに新しいコマンドが生成されるはずです。おそらく、ロボットを地面に置き、障害物に到達したときの動作を確認する事ができるでしょう。

If you want to stop this behavior, you can unattach this callback by executing the code below.

(訳)もし、今の挙動を止めたい場合は、下記コードを実行し、このカールバックの関連付けをやめます。

In [ ]:
camera.unobserve(update, names='value')
robot.stop()

Perhaps you want the robot to run without streaming video to the browser.  You can unlink the camera as below.

(訳) もしかしたら、videoをブラウザに転送しないでロボットを実行したいと思いますが、その場合は、下記のようにcameraをunkiklｋしてください。

In [ ]:
camera_link.unlink()  # don't stream to browser (will still run camera)

To continue streaming call the following.

(訳) streamingを続ける場合は、下記コードで続けます。

In [ ]:
camera_link.link()  # stream to browser (wont run camera)

### Conclusion

That's it for this live demo!  Hopefully you had some fun and your robot avoided collisions intelligently! 

(訳) 以上がLive demoです。うまく行けば、ロボットが衝突をインテリジェントに回避できたことと思います。

If your robot wasn't avoiding collisions very well, try to spot where it fails.  The beauty is that we can collect more data for these failure scenari
and the robot should get even better :)

(訳) avoiding collisionsが上手く行かない場合、失敗した箇所を見つけてください。美しさは、これら上手くいかないシナリオについてより多くのデータを収集すれば、ロボットはさらに良くなるはずです